# Strategic Site Selection: Leveraging Geospatial Analytics at Scale
@Luis Gerardo Baeza


In industries with physical footprints—such as banking and retail—deciding where to open a new branch isn't just a "gut feeling" anymore; it's a data-driven science. By analyzing transactional data and geographic patterns, companies can transform raw coordinates into strategic assets.

**Core Objectives**
- Identify Demand: Map transaction "hotspots" to find where customers need a physical presence.
- Prevent Cannibalization: Ensure a new location captures new market share instead of just stealing traffic from your existing branches.
- Geospatial Intelligence: Use Google Maps for high-precision geocoding and BigQuery’s GEOGRAPHY functions to analyze spatial relationships at scale.

**Why this Workflow?**
- Speed: Process millions of transactions in seconds.
- Scale: Analyze entire cities or countries simultaneously.
- Accuracy: Use real-world coordinates to minimize the risk of expensive real estate mistakes.

## 1. Setup Google Maps
### 1.1 Dependencies

In [ ]:
pip install -q googlemaps

  Preparing metadata (setup.py) ... done


### 1.2 Imports

In [13]:
import os
import sys
import pandas as pd
# Maps
import googlemaps
from typing import Iterator
from pyspark.sql.functions import pandas_udf, col, expr
# PySpark
from google.cloud.dataproc_spark_connect import DataprocSparkSession
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType

spark = DataprocSparkSession.builder \
    .getOrCreate()
spark.addArtifacts("googlemaps", pypi=True)

sc-20260213-163237-qhkzgg Dataproc Session is not active, stopping and creating a new one


█████████████████████████████████████████████████▌                              

### 1.3 Variables

In [ ]:
api_key = ""
project_id = ""
location_id = "us-central1"
INPUT_TABLE = f"{project_id}.retail_geodemo.retail_sale_point"
OUTPUT_TABLE = f"{project_id}.retail_geodemo.retail_sale_point_enriched"

## 2. Prepare sample data

In [ ]:
%%bigquery
CREATE SCHEMA IF NOT EXISTS retail_geodemo;

Query is running:   0%|          |

""


In [ ]:
%%bigquery
CREATE OR REPLACE TABLE `retail_geodemo.retail_areas` AS
WITH raw_areas AS (
  SELECT 1 as id, 'STGOC' as number_area, ST_GEOGFROMTEXT('MULTIPOLYGON(((-70.655 -33.448, -70.650 -33.448, -70.650 -33.452, -70.655 -33.452, -70.655 -33.448)))') as area_polygon
  UNION ALL
  SELECT 2, 'PROVI', ST_GEOGFROMTEXT('MULTIPOLYGON(((-70.615 -33.428, -70.610 -33.428, -70.610 -33.432, -70.615 -33.432, -70.615 -33.428)))')
  UNION ALL
  SELECT 3, 'LCONC', ST_GEOGFROMTEXT('MULTIPOLYGON(((-70.580 -33.400, -70.570 -33.400, -70.570 -33.410, -70.580 -33.410, -70.580 -33.400)))')
  UNION ALL
  SELECT 4, 'MAIPU', ST_GEOGFROMTEXT('MULTIPOLYGON(((-70.760 -33.500, -70.750 -33.500, -70.750 -33.510, -70.760 -33.510, -70.760 -33.500)))')
  UNION ALL
  SELECT 5, 'PALTO', ST_GEOGFROMTEXT('MULTIPOLYGON(((-70.580 -33.580, -70.570 -33.580, -70.570 -33.590, -70.580 -33.590, -70.580 -33.580)))')
  UNION ALL
  -- Generación de 45 áreas adicionales aleatorias en Chile (Rango lat -18 a -53)
  SELECT
    n,
    RPAD(UPPER(SUBSTR(CAST(GENERATE_UUID() AS STRING), 1, 5)), 5, 'X'),
    ST_GEOGFROMTEXT(FORMAT('MULTIPOLYGON(((%.3f %.3f, %.3f %.3f, %.3f %.3f, %.3f %.3f, %.3f %.3f)))',
      -71.0 - (n*0.1), -30.0 - (n*0.2),
      -70.9 - (n*0.1), -30.0 - (n*0.2),
      -70.9 - (n*0.1), -30.1 - (n*0.2),
      -71.0 - (n*0.1), -30.1 - (n*0.2),
      -71.0 - (n*0.1), -30.0 - (n*0.2)))
  FROM UNNEST(GENERATE_ARRAY(6, 50)) AS n
)
SELECT * FROM raw_areas;

Query is running:   0%|          |

""


In [ ]:
%%bigquery
SELECT * FROM retail_geodemo.retail_areas;

Query is running:   0%|          |

Downloading:   0%|          |

,id,number_area,area_polygon
0,26,136F4,"POLYGON((-73.6 -35.3, -73.5 -35.3, -73.5 -35.2..."
1,31,2F35D,"POLYGON((-74.1 -36.3, -74 -36.3, -74 -36.2, -7..."
2,44,4B1F1,"POLYGON((-75.4 -38.9, -75.3 -38.9, -75.3 -38.8..."
3,38,BAF60,"POLYGON((-74.8 -37.7, -74.7 -37.7, -74.7 -37.6..."
4,1,STGOC,"POLYGON((-70.655 -33.452, -70.65 -33.452, -70...."
5,18,B83AE,"POLYGON((-72.8 -33.7, -72.7 -33.7, -72.7 -33.6..."
6,46,057AD,"POLYGON((-75.6 -39.3, -75.5 -39.3, -75.5 -39.2..."
7,19,82066,"POLYGON((-72.9 -33.9, -72.8 -33.9, -72.8 -33.8..."
8,13,8A2D4,"POLYGON((-72.3 -32.7, -72.2 -32.7, -72.2 -32.6..."
9,25,B2D84,"POLYGON((-73.5 -35.1, -73.4 -35.1, -73.4 -35, ..."


Podemos visualizar las areas con [BigQuery GeoViz](https://bigquerygeoviz.appspot.com/)

![areas_polygons.png](areas_polygons.png)

In [ ]:
%%bigquery
CREATE OR REPLACE TABLE `retail_geodemo.retail_sale_point` AS
WITH seed_data AS (
  -- Usamos comillas dobles para evitar el error con O'Higgins
  SELECT "Avenida Libertador Bernardo O'Higgins" as calle, "Santiago" as comuna, "Región Metropolitana" as region UNION ALL
  SELECT "Calle Vicuña Mackenna", "La Florida", "Región Metropolitana" UNION ALL
  SELECT "Avenida Apoquindo", "Las Condes", "Región Metropolitana" UNION ALL
  SELECT "Avenida Pajaritos", "Maipú", "Región Metropolitana" UNION ALL
  SELECT "Calle Prat", "Antofagasta", "Antofagasta" UNION ALL
  SELECT "Avenida Libertad", "Viña del Mar", "Valparaíso" UNION ALL
  SELECT "Calle Diego Portales", "Temuco", "Araucanía" UNION ALL
  SELECT "Avenida Alemania", "Concepción", "Biobío" UNION ALL
  SELECT "Calle Baquedano", "Iquique", "Tarapacá" UNION ALL
  SELECT "Avenida Costanera", "Puerto Montt", "Los Lagos"
),
generator AS (
  SELECT n FROM UNNEST(GENERATE_ARRAY(1, 2000)) n
),
numbered_seeds AS (
  SELECT *, ROW_NUMBER() OVER() as seed_id FROM seed_data
),
total_seeds AS (
  SELECT COUNT(*) as cnt FROM seed_data
)
SELECT
  GENERATE_UUID() as id,
  -- Combinamos calle + espacio + número aleatorio
  CONCAT(s.calle, " ", CAST(FLOOR(RAND() * 9999) + 1 AS INT64)) as direccion,
  s.comuna,
  s.region
FROM generator g
CROSS JOIN total_seeds t
JOIN numbered_seeds s
  ON MOD(g.n, t.cnt) + 1 = s.seed_id;

Query is running:   0%|          |

""


In [ ]:
%%bigquery
SELECT * FROM retail_geodemo.retail_sale_point LIMIT 50;

Query is running:   0%|          |

Downloading:   0%|          |

,id,direccion,comuna,region
0,10e7697e-f80c-4859-8784-bb64f2a43f9a,Calle Baquedano 4236,Iquique,Tarapacá
1,2f63e2af-8d05-4692-bf5e-0024d8280bc5,Avenida Libertad 1638,Viña del Mar,Valparaíso
2,ec76e800-901b-46df-88b1-5be40956810c,Calle Baquedano 9081,Iquique,Tarapacá
3,4e6d84b4-1cfd-40b9-bc53-e22e866b3829,Calle Baquedano 1708,Iquique,Tarapacá
4,ab101f4c-55f4-4ae6-a7ca-b97bbcde2027,Calle Prat 3833,Antofagasta,Antofagasta
5,2d873b44-3161-4339-a69d-41c485241cb1,Calle Prat 8186,Antofagasta,Antofagasta
6,af03f38b-4733-47d9-971b-1978c6d88a6a,Avenida Apoquindo 6204,Las Condes,Región Metropolitana
7,04f5c6ab-c33c-471b-8555-007777c835a6,Calle Diego Portales 3106,Temuco,Araucanía
8,6bd5cbe3-0321-4928-8a92-1ab98bed4148,Avenida Alemania 5588,Concepción,Biobío
9,ae8ebe0d-628c-42a8-9511-bcbfe669c772,Calle Diego Portales 6709,Temuco,Araucanía


##3. Maps Geocoding


In [ ]:
def geocode_address(address):
  gmaps = googlemaps.Client(key=api_key)
  geocode_result = gmaps.geocode(address)
  latitude = geocode_result[0]['geometry']['location']['lat']
  longitude = geocode_result[0]['geometry']['location']['lng']

  return latitude, longitude

In [ ]:
latitude, longitude = geocode_address(address = "Cerro el plomo 5680")
print(f"Latitude: {latitude}, Longitude: {longitude}")

Latitude: -33.4053637, Longitude: -70.5737473


##3. PySpark processing at scale

In [ ]:
@pandas_udf("string")
def geocode_udf(batch_iter: Iterator[pd.Series]) -> Iterator[pd.Series]:
    client = googlemaps.Client(key=api_key)
    for batch in batch_iter:
        yield batch.apply(lambda addr:
            f"POINT({client.geocode(addr)[0]['geometry']['location']['lng']} {client.geocode(addr)[0]['geometry']['location']['lat']})"
            if client.geocode(addr) else None)

In [ ]:
df_prepared = spark.read.format("bigquery") \
    .option("query", f"""
        SELECT *, CONCAT(direccion, ', ', comuna, ', ', region, ', Chile') as full_address
        FROM `{project_id}.retail_geodemo.retail_sale_point`
    """).load()

In [ ]:
df_intermediate = df_prepared.withColumn("wkt", geocode_udf(col("full_address")))
df_intermediate.show()

+--------------------+--------------------+------------+--------------------+--------------------+--------------------+
|                  id|           direccion|      comuna|              region|        full_address|                 wkt|
+--------------------+--------------------+------------+--------------------+--------------------+--------------------+
|10e7697e-f80c-485...|Calle Baquedano 4236|     Iquique|            Tarapacá|Calle Baquedano 4...|POINT(-70.1521233...|
|2f63e2af-8d05-469...|Avenida Libertad ...|Viña del Mar|          Valparaíso|Avenida Libertad ...|POINT(-71.5485564...|
|ec76e800-901b-46d...|Calle Baquedano 9081|     Iquique|            Tarapacá|Calle Baquedano 9...|POINT(-70.1521233...|
|4e6d84b4-1cfd-40b...|Calle Baquedano 1708|     Iquique|            Tarapacá|Calle Baquedano 1...|POINT(-70.1521542...|
|ab101f4c-55f4-4ae...|     Calle Prat 3833| Antofagasta|         Antofagasta|Calle Prat 3833, ...|POINT(-70.3939298...|
|2d873b44-3161-433...|     Calle Prat 81

In [ ]:
# 6. Write back to BigQuery
# 'indirect' write method via GCS is recommended for massive volumes (> 10GB)
# If using 'direct', ensure your project has enough BigQuery Write API quota
df_intermediate.write.format("bigquery") \
    .option("table", OUTPUT_TABLE) \
    .option("writeMethod", "direct") \
    .mode("overwrite") \
    .save()

print(f"Successfully processed and saved to {OUTPUT_TABLE}")

Successfully processed and saved to lgbaeza-202310.retail_geodemo.retail_sale_point_enriched


In [14]:
%%bigquery
# Save with the right format
CREATE OR REPLACE TABLE retail_geodemo.retail_sale_point_enriched_g as
select * except(wkt), ST_GEOGFROMTEXT(wkt) AS geopoint_location from retail_geodemo.retail_sale_point_enriched

Query is running:   0%|          |

""


## 4. Finding closes areas of analysis to the data points

In [26]:
%%bigquery
CREATE OR REPLACE TABLE `retail_geodemo.point_to_nearest_area` AS
WITH distances AS (
  SELECT
    p.id AS point_id,
    p.direccion,
    a.number_area,
    p.geopoint_location,
    -- ST_DISTANCE calcula la distancia mínima en metros entre el punto y el borde del polígono
    ST_DISTANCE(p.geopoint_location, a.area_polygon) AS distance_meters
  FROM
    `retail_geodemo.retail_sale_point_enriched_g` AS p
  CROSS JOIN
    `retail_geodemo.retail_areas` AS a
)
SELECT
  * except (ranking)
FROM (
  SELECT
    *,
    -- Numeramos las filas por cada punto, ordenadas por la distancia más corta
    ROW_NUMBER() OVER(PARTITION BY point_id ORDER BY distance_meters ASC) as ranking
  FROM distances
)
-- Filtramos para obtener solo el registro #1 (el más cercano)
WHERE ranking = 1;

Query is running:   0%|          |

""


In [27]:
%%bigquery
select
  point_id, direccion, p.number_area,
  a.area_polygon, p.geopoint_location
from retail_geodemo.point_to_nearest_area  p
INNER JOIN retail_geodemo.retail_areas a using (number_area)
limit 10

Query is running:   0%|          |

Downloading:   0%|          |

,point_id,direccion,number_area,area_polygon,geopoint_location
0,7f64c377-0017-4747-a700-65f41789c78b,Avenida Libertad 1732,8A2D4,"POLYGON((-72.3 -32.7, -72.2 -32.7, -72.2 -32.6...",POINT(-71.5485564 -33.0081471)
1,183b904d-21c0-46e4-8c0e-8d0c1a3c7827,Avenida Libertad 2056,8A2D4,"POLYGON((-72.3 -32.7, -72.2 -32.7, -72.2 -32.6...",POINT(-71.5485564 -33.0081471)
2,3cb64646-3a64-4b34-bdcb-2d8ce4685c9b,Avenida Libertad 1748,8A2D4,"POLYGON((-72.3 -32.7, -72.2 -32.7, -72.2 -32.6...",POINT(-71.5485564 -33.0081471)
3,8bb3ce53-c944-4201-8ffa-294b432d0f05,Avenida Libertad 2630,8A2D4,"POLYGON((-72.3 -32.7, -72.2 -32.7, -72.2 -32.6...",POINT(-71.5485564 -33.0081471)
4,6b77c477-43a3-46d2-b040-33b3b9b31514,Avenida Libertad 1644,8A2D4,"POLYGON((-72.3 -32.7, -72.2 -32.7, -72.2 -32.6...",POINT(-71.5485564 -33.0081471)
5,f4a2d2d4-5c59-4261-8242-2e5b4168c14e,Avenida Libertad 1838,8A2D4,"POLYGON((-72.3 -32.7, -72.2 -32.7, -72.2 -32.6...",POINT(-71.5485564 -33.0081471)
6,3be6d85f-107b-42ad-bde2-0baa9254c40a,Avenida Libertad 2536,8A2D4,"POLYGON((-72.3 -32.7, -72.2 -32.7, -72.2 -32.6...",POINT(-71.5485564 -33.0081471)
7,58701e2c-0aa5-4de8-9682-8857fea88bec,Avenida Libertad 2026,8A2D4,"POLYGON((-72.3 -32.7, -72.2 -32.7, -72.2 -32.6...",POINT(-71.5485564 -33.0081471)
8,5964fa07-c74d-46a7-899f-0c0c6d5f8380,Avenida Libertad 1890,8A2D4,"POLYGON((-72.3 -32.7, -72.2 -32.7, -72.2 -32.6...",POINT(-71.5485564 -33.0081471)
9,ed52d73a-85a2-4950-bd17-941a9cc30b1b,Avenida Libertad 2366,8A2D4,"POLYGON((-72.3 -32.7, -72.2 -32.7, -72.2 -32.6...",POINT(-71.5485564 -33.0081471)


We create a table for visualizing both areas and points.

In [32]:
%%bigquery
create or replace table retail_geodemo.retail_sale_point_areas as
with analysis_dedup as (
  with analysis as (
  select
    point_id, direccion, p.number_area,
    ST_AsText(a.area_polygon) area_polygon, ST_AsText(p.geopoint_location) geopoint_location
  from retail_geodemo.point_to_nearest_area  p
  INNER JOIN retail_geodemo.retail_areas a using (number_area)
  )
  select number_area, area_polygon from analysis
  union all
  select number_area, geopoint_location from analysis group by 1,2
)
select number_area, ST_GEOGFROMTEXT(area_polygon) area_polygon from analysis_dedup


Query is running:   0%|          |

""


### 4.1 Visual analysis
We can see how the data poings are groupped into the closest locations defined by us.

![polygons_grouped.png](polygons_grouped.png)

## Lastly: What's Next
* The cross join can be optimized with other methods for large scale scenarios
* Further analysis can be made other than only the closes area.
* Additional data can be brought into the analysis using BigQuery Geospatial Analytics functions. If you are a wine fan, you can find an example about geographic feature analysis with a sample agricultural use case as part of the [BigQuery Data Science Book.](https://github.com/GoogleCloudPlatform/ai-ml-recipes/blob/main/notebooks/analytics/assessing_risks_geospatial_bqml.ipynb)